In [6]:
%pip install fastai
%pip install torch torchvision torchaudio


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/9b/f5/d3c9b0ba802e2b8e363c5366b37b6b5fb703ac052a5afc334c36b255401f/torchaudio-2.4.1-cp312-cp312-win_amd64.whl.metadata
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
    --------------------------------------- 0.0/2.4 MB 330.3 kB/s eta 0:00:08
   - -------------------------------------- 0.1/2.4 MB 751.6 kB/s eta 0:00:04
   ------- -------------------------------- 0.4/2.4 MB 2.7 MB/s eta 0:00:01
   --------------------------- ------------ 1.6/2.4 MB 8.1 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 9.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from fastai.vision.all import *
import pathlib

# Stel het pad in naar je afbeeldingen
data_dir = pathlib.Path("/Screenshots")

# DataBlock voor het verwerken van afbeeldingen
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),  # We classificeren afbeeldingen
    get_items=get_image_files,           # Alle afbeeldingsbestanden ophalen
    splitter=RandomSplitter(valid_pct=0.2, seed=123),  # 20% voor validatie
    get_y=parent_label,                  # Labels worden gebaseerd op mappenamen
    item_tfms=Resize(180),               # Afbeeldingen worden geschaald naar 180x180
    batch_tfms=aug_transforms(size=180)  # Basisaugmentatie en schaling
)

# Maak een DataLoaders object
dls = dblock.dataloaders(data_dir, bs=32)

# Het model trainen met een vooraf getrainde ResNet-architectuur
learn = cnn_learner(dls, resnet18, metrics=accuracy)

# Samenvatting van het model
learn.model

# Optionele callback voor vroegtijdig stoppen
early_stopping_cb = EarlyStoppingCallback(monitor='valid_loss', patience=8)

# Train het model
learn.fit_one_cycle(20, cbs=[early_stopping_cb])

# Bewaar het getrainde model
learn.export('ad-model.h5')


TypeError: 'NoneType' object is not iterable